In [ ]:
import sys
if "google.colab" in sys.modules:
    print("Colab Detected")
    import tensorflow as tf
    gpus = tf.config.list_physical_devices("GPU")
    if not gpus:
        raise RuntimeError("Nessuna GPU trovata.Controlla di aver selezionato il runtime giusto.")
    else:
        print(f"Trovate {len(gpus)} GPU:\n{gpus}")

    !git clone https://github.com/AtomicDonuts/Progetto_Computings.git
    %cd Progetto_Computings/
    !git checkout modell
    !pip install -q -r requirements.txt
    !python3 fits_import/fits2csv.py

    sys.path.append("imports/")
    import custom_variables as custom_paths
    import ann_models as ann
else:
    print("Local Machine Detected")
    sys.path.append("../imports/")
    import custom_variables as custom_paths
    import ann_models as ann

In [ ]:
import numpy as np
import pandas as pd
import keras
import keras_tuner as kt
from scipy import stats
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
df = pd.read_csv(custom_paths.csv_path)
df = df[(df["CLASS_GENERIC"] == "AGN") | (df["CLASS_GENERIC"] == "Pulsar")]
print(f"Dataset Dimentions: {len(df)}")

df["PowerLaw"] = np.where(df["SpectrumType"] == "PowerLaw",1,0,)
df["LogParabola"] = np.where(df["SpectrumType"] == "LogParabola",1,0,)
df["PLSuperExpCutoff"] = np.where(df["SpectrumType"] == "PLSuperExpCutoff",1,0,)

norm_cols = ["GLAT", "PowerLaw","LogParabola","PLSuperExpCutoff" ,"Variability_Index"]
input_datas = df[norm_cols].to_numpy()

is_agn = df["CLASS_GENERIC"].to_numpy() == "AGN"
is_psr = df["CLASS_GENERIC"].to_numpy() == "Pulsar"
labels = np.zeros((len(df)), dtype=int)
labels[is_agn] = 0
labels[is_psr] = 1

In [ ]:
num_splits = 10
skf = StratifiedKFold(n_splits=num_splits, shuffle=True)
fold_index = 1
for a, b in skf.split(input_datas, labels):
    train_inp = input_datas[a]
    train_lab = labels[a]
    val_inp = input_datas[b]
    val_lab = labels[b]
    tuner = kt.Hyperband(
        ann.hp_model,
        objective="accuracy",
        max_epochs=100,
        factor=3,
        overwrite=False,
        directory="DNN",
        project_name=f"Fold_{fold_index}",
    )
    stop_early = keras.callbacks.EarlyStopping(monitor="accuracy", patience=5)
    tuner.search(
        train_inp,
        train_lab,
        epochs=50,
        validation_data=(val_inp, val_lab),
        callbacks=[stop_early],
    )
    fold_index += 1

In [ ]:
tuners = []
for i in range(1,num_splits+1):
    tuners.append(kt.Hyperband(
        ann.hp_model,
        objective="accuracy",
        max_epochs=100,
        factor=3,
        overwrite=False,
        directory="DNN",
        project_name=f"Fold_{i}",
    ))

In [ ]:
nodes = []
dropout = []
for j in tuners:
    for i in j.get_best_hyperparameters(num_trials=3):
        nodes.append(i.get("nodes"))
        dropout.append(i.get("dropout"))
modal_nodes = stats.mode(nodes).mode
modal_dropout = stats.mode(dropout).mode

print(f"Modal Nodes: {modal_nodes}")
print(f"Modal Dropout: {modal_dropout}")

best_matching_tuner = None
highest_accuracy_matching_tuner = -1
best_matching_tuner_index = -1

print("Ricerca del tuner con iperparametri modali e massima accuratezza...")

for index, tuner_obj in enumerate(tuners):
    
    # Ottieni i migliori trial dal tuner
    # tuner.oracle.get_best_trials() restituisce una lista di oggetti Trial
    best_trial = tuner_obj.oracle.get_best_trials(num_trials=1)[0]
    best_hps = best_trial.hyperparameters
    current_accuracy = best_trial.score

    current_nodes = best_hps.get("nodes")
    current_dropout = best_hps.get("dropout")

    # Controlla se i valori corrispondono alla moda
    if current_nodes == modal_nodes and np.isclose(current_dropout, modal_dropout):
        print(
            f"  Fold {index + 1} corrisponde. Accuratezza: {current_accuracy:.4f}"
        )
        # Se corrisponde e ha un'accuratezza maggiore, aggiorna il miglior tuner
        if current_accuracy > highest_accuracy_matching_tuner:
            highest_accuracy_matching_tuner = current_accuracy
            best_matching_tuner = tuner_obj
            best_matching_tuner_index = index

if best_matching_tuner:
    print(
        f"\nIl miglior tuner trovato con NODES={modal_nodes} e DROPOUT={modal_dropout} è per la Fold {best_matching_tuner_index + 1} con Accuratezza: {highest_accuracy_matching_tuner:.4f}"
    )
    print("I suoi migliori iperparametri sono:\n")
    best_matching_tuner.results_summary(num_trials=1)
    best_model = best_matching_tuner.get_best_models(num_models=1)[0]
    # print("Sommario del miglior modello:\n")
    # best_model.summary()
else:
    print(
        "Nessun tuner trovato con iperparametri che corrispondono esattamente ai valori della moda."
    )

In [ ]:
early_stopping = EarlyStopping(
    monitor="accuracy", patience=10, restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(monitor="accuracy", factor=0.5, patience=5)
best_model.optimizer.learning_rate = 0.01
history = best_model.fit(
    input_datas,
    labels,
    epochs=300,
    callbacks=[early_stopping, reduce_lr],
)